In [ ]:
#Case Study - Minimizing Data Center Energy Consumption Costs
#Artificial Intelligence Training

#libraries
import os
import numpy as np
import random as rn
import Enviroment
import Brain
import DQL

In [ ]:
#Seed Configuration for Reproducibility
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)

#PARAMETER SETTING
epsilon = .3
number_actions = 5
direction_boundary = (number_actions - 1) / 2
number_epochs = 100
max_memory = 3000
batch_size = 512
temperature_step = 1.5

# ENVIRONMENT CREATION
env = environment.Environment(optimal_temperature = (18.0, 24.0), initial_month = 0,
                              initial_number_users = 20, initial_rate_data = 30)

# CREATION OF THE BRAIN (NEURAL NETWORK)
brain = brain.Brain(learning_rate = 0.00001, number_actions = number_actions)

# CREATION OF DEEP Q-LEARNING
dqn = dqn.DQN(max_memory = max_memory, discount = 0.9)

# SETTING THE TRAINING/TEST MODE
train = True

# ARTIFICIAL INTELLIGENCE TRAINING
env.train = train
model = brain.model
early_stopping = True
patience = 10
best_total_reward = -np.inf
patience_count = 0
if (env.train):
    # STARTING THE LOOP THAT WILL RUN THROUGH ALL SEASONS (1 SEASON = 5 MONTHS)
    for epoch in range(1, number_epochs):
        # INITIALIZING ENVIRONMENT AND LOOP VARIABLES
        total_reward = 0
        loss = 0.
        new_month = np.random.randint(0, 12)
        env.reset(new_month = new_month)
        game_over = False
        current_state, _, _ = env.observe()
        timestep = 0
        
        # STARTING THE LOOP THAT WILL CYCLE THROUGH EACH TIMESTEP (1 TIMESTEP = 1 MINUTE)
        while ((not game_over) and timestep <= 5 * 30 * 24 * 60):
            # EXECUTION OF THE NEXT ACTION WITH EXPLOITATION
            if np.random.rand() <= epsilon:
                action = np.random.randint(0, number_actions)
                if (action - direction_boundary < 0):
                    direction = -1
                else:
                    direction = 1
                energy_ai = abs(action - direction_boundary) * temperature_step
            
            # EXECUTION OF THE NEXT ACTION BY INFERENCE, USING THE NEURAL NETWORK
            else:
                q_values = model.predict(current_state)
                action = np.argmax(q_values[0])
                if (action - direction_boundary < 0):
                    direction = -1
                else:
                    direction = 1
                energy_ai = abs(action - direction_boundary) * temperature_step
            
            # UPDATING THE ENVIRONMENT AND GETTING THE NEW STATE
            next_state, reward, game_over = env.update_env(direction, energy_ai, int(timestep / (30 * 24 * 60)))
            total_reward += reward
            
            # STORING THE NEW TRANSACTION IN MEMORY
            dqn.remember([current_state, action, reward, next_state], game_over)
            
            # SEPARATION INTO TWO SEPARATE BATCHES (INPUTS AND TARGETS)
            inputs, targets = dqn.get_batch(model, batch_size = batch_size)
            
            # LOSS CALCULATION USING BATCHES
            loss += model.train_on_batch(inputs, targets)
            timestep += 1
            current_state = next_state
        
        # PRINTOUT OF THE TRAINING RESULTS FOR EACH SEASON
        print('\n')
        print("Epoch: {:03d}/{:03d}".format(epoch, number_epochs))
        print("Total energy spent with AI: {:.0f}".format(env.total_energy_ai))
        print("Total energy spent with no AI: {:.0f}".format(env.total_energy_noai))        
        
        # EARLY STOPPING
        if (early_stopping):
            if (total_reward <= best_total_reward):
                patience_count += 1
            elif (total_reward > best_total_reward):
                best_total_reward = total_reward
                patience_count = 0
            if (patience_count > patience):
                print("Early stopping")
                break
        
        # SAVE THE MODEL
        model.save("model.h5")


